# Otras funcionalidades de Julia

### Funciones Mutables vs No Mutables 

Por convención, funciones seguidas de `!` alteran sus contenidos y funciones que no lo tienen no pueden ser modificadas.

In [1]:
v = [3, 5, 2]

3-element Vector{Int64}:
 3
 5
 2

In [2]:
sort(v)

3-element Vector{Int64}:
 2
 3
 5

In [3]:
v

3-element Vector{Int64}:
 3
 5
 2

`sort(v)` devuelve un arrego ordenado que contiene los mismos elementos de `v` sin alterar a `v`.


In [4]:
sort!(v)

3-element Vector{Int64}:
 2
 3
 5

In [5]:
v

3-element Vector{Int64}:
 2
 3
 5

`sort!(v)` altera el argumento `v` que se le pasa a la función.

## Funciones de orden superior 

La función `map()` es una función en Julia que toma como uno de sus argumentos de entrada una _función_. `map()` aplica la función a cada elemento de la estructura de datos que se le pasa.

Por ejemplo, al ejecutar:
```julia
map(f,[1, 2, 3])
```
obtendremos un arreglo de salida donde la función `f()` se aplica a cada uno de los elementos del array `[1, 2, 3]`, es decir,
```julia
[f(1), f(2), f(3)]
```

Se han elevado al cuadrado todos los elementos de `[1, 2, 3]` en lugar de elevar al cuadrado el vector mismo. Para hacer esto, se puede pasar a la función `map()` una función anónima en lugar del nombre de la función: 

In [6]:
g = x -> x^2
g(5)

25

In [7]:
map(g, [1,2,3])

3-element Vector{Int64}:
 1
 4
 9

In [8]:
f = x -> x^3

#3 (generic function with 1 method)

In [9]:
map(x -> x^3, [1, 2, 3])

3-element Vector{Int64}:
  1
  8
 27

### Broadcast 

`broadcast` es otra función de orden superior como `map`. `broadcast` es una generalización de `map`, por lo que puede hacer cualquier cosa que hace `map` e incluso algo más. La sintáxis de `broadcast` es la misma que la de `map`.

In [10]:
broadcast(f, [1, 2, 3])

3-element Vector{Int64}:
  1
  8
 27

Una vez más se ha aplicado `f` a todos los elementos de `[1, 2, 3]`, esta vez difundiendo (_broadcasting_) la función `f`.
Un poco de "syntactic sugar" para llamar a `broadcast` es colocar un `.` entre el nombre de la función que se desee hacer `brodcast` y sus argumentos de entrada, por ejemplo:
```julia
broadcast(f,[1, 2, 3])
```
es lo mismo que escribir,
```julia
f.([1, 2, 3])
```

In [11]:
f.([1, 2, 3])

3-element Vector{Int64}:
  1
  8
 27

In [12]:
@time broadcast(f,[1, 2, 3])

  0.000005 seconds (2 allocations: 224 bytes)


3-element Vector{Int64}:
  1
  8
 27

In [13]:
@time f.([1, 2, 3])

  0.000020 seconds (4 allocations: 256 bytes)


3-element Vector{Int64}:
  1
  8
 27

Notemos la diferencia entre hacer la llamada a la función:
```julia 
f([1, 2, 3])
```
y hacer el _broadcast_ de la función.

Es posible elevar al cuadrado cada elemento del vector, pero no es posible elevar al cuadrado al vector.

In [14]:
f([1, 2, 3])

LoadError: MethodError: no method matching ^(::Vector{Int64}, ::Int64)
[0mClosest candidates are:
[0m  ^([91m::Union{AbstractChar, AbstractString}[39m, ::Integer) at strings/basic.jl:718
[0m  ^([91m::LinearAlgebra.UniformScaling[39m, ::Number) at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.6/LinearAlgebra/src/uniformscaling.jl:298
[0m  ^([91m::LinearAlgebra.Hermitian[39m, ::Integer) at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.6/LinearAlgebra/src/symmetric.jl:890
[0m  ...

Entiéndase la operación de **Broadcast** como una "expansión la dimensión unaria" de los objetos que se pasan a la función.

Para hacerlo más claro, apliquemos `f()` a una matriz `A` y veamos la diferencia al hacer un _broadcast_ a `f()` sobre `A`

In [15]:
A = [i + 3*j for j in 0:2, i in 1:3]

3×3 Matrix{Int64}:
 1  2  3
 4  5  6
 7  8  9

In [16]:
f(A)  # multiplica A por si misma 3 veces

3×3 Matrix{Int64}:
  468   576   684
 1062  1305  1548
 1656  2034  2412

In [17]:
f.(A)

3×3 Matrix{Int64}:
   1    8   27
  64  125  216
 343  512  729

Con el _broadcast_ cada elemento de `A` se elevan al cuadrado.

La notación con `.` para hacer el _broadcast_ permite escribir composiciones complejas de expresiones en una manera que parece más natural y cercana a la notación matemática convencional.

In [18]:
A

3×3 Matrix{Int64}:
 1  2  3
 4  5  6
 7  8  9

Podemos escribir

In [19]:
A .+ 2 .* f.(A) ./ A   

3×3 Matrix{Float64}:
   3.0   10.0   21.0
  36.0   55.0   78.0
 105.0  136.0  171.0

que será mucho más natural que escribir:

In [20]:
broadcast(x -> x + 2 * f(x) / x, A)

3×3 Matrix{Float64}:
   3.0   10.0   21.0
  36.0   55.0   78.0
 105.0  136.0  171.0

**Ejemplo:**

Evaluación de varias funciones sobre un arreglo de números aleatorios

In [21]:
funs = [sin,cos]

2-element Vector{Function}:
 sin (generic function with 13 methods)
 cos (generic function with 13 methods)

In [22]:
vals = rand(5)

5-element Vector{Float64}:
 0.4259716128953428
 0.28988107378368766
 0.5768431504855278
 0.08284143956124623
 0.4787151448966338

In [23]:
[[f(v) for f in funs] for v in vals]

5-element Vector{Vector{Float64}}:
 [0.413205763963335, 0.9106376868038554]
 [0.2858382627645102, 0.9582778759523601]
 [0.545380623749002, 0.8381885081765017]
 [0.08274671935432011, 0.9965706098596815]
 [0.4606391347372261, 0.887587509797507]

In [24]:
map.(funs, Ref(vals))

2-element Vector{Vector{Float64}}:
 [0.413205763963335, 0.2858382627645102, 0.545380623749002, 0.08274671935432011, 0.4606391347372261]
 [0.9106376868038554, 0.9582778759523601, 0.8381885081765017, 0.9965706098596815, 0.887587509797507]

In [25]:
opera(f, x) = f(x)
opera.(funs, permutedims(vals))

2×5 Matrix{Float64}:
 0.413206  0.285838  0.545381  0.0827467  0.460639
 0.910638  0.958278  0.838189  0.996571   0.887588

## Interoperabilidad

### Interoperabilidad con Python: `PyCall`

Tomando en cuenta el ecosistema de paquetes de Python, la gama de paquetes disponibles en Julia puede parecer un tanto limitada. Sin embargo, esto se compensa con la facilidad de llamar a paquetes escritos en otros idiomas desde Julia.

In [26]:
using Pkg
Pkg.add("PyCall")

using PyCall

    Updating registry at `~/.julia/registries/General`
    Updating git-repo `https://github.com/JuliaRegistries/General.git`
   Resolving package versions...
   Installed MacroTools ─ v0.5.7
   Installed PyCall ───── v1.92.3
    Updating `~/.julia/environments/v1.6/Project.toml`
  [438e738f] + PyCall v1.92.3
    Updating `~/.julia/environments/v1.6/Manifest.toml`
  [1914dd2f] + MacroTools v0.5.7
  [438e738f] + PyCall v1.92.3
  [37e2e46d] + LinearAlgebra
    Building PyCall → `~/.julia/scratchspaces/44cfe95a-1eb2-52ea-b672-e2afdf69b78f/169bb8ea6b1b143c5cf57df6d34d022a7b60c6db/build.log`
Precompiling project...
  ✓ MacroTools
  ✓ PyCall
  2 dependencies successfully precompiled in 14 seconds (15 already precompiled)


`PyCall` tiene una interfase de alto nivel diseñada para el transporte de código entre Julia y Python y es transparente desde el punto de vista del usuario. Por ejemplo, para importar el mótudlo `math` de Python utilizamos el método `pyimport()`:

In [27]:
pymath = pyimport("math")

PyObject <module 'math' from '/home/oscar/anaconda3/lib/python3.6/lib-dynload/math.cpython-36m-x86_64-linux-gnu.so'>

In [28]:
pymath.sin(1)

0.8414709848078965

In [29]:
pymath.sin(0.3 * pymath.pi) - sin(0.3 * pymath.pi)

0.0

In [30]:
nprandom = pyimport("numpy.random")

PyObject <module 'numpy.random' from '/home/oscar/anaconda3/lib/python3.6/site-packages/numpy/random/__init__.py'>

In [31]:
nprandom.rand(3,4)

3×4 Matrix{Float64}:
 0.412271   0.0970202  0.729396  0.109079
 0.0529896  0.521312   0.726257  0.71241
 0.646229   0.862924   0.866118  0.957738

Al hacer una llamada a `arrays` de  `numpy` estos son convertidos automáticamente a arreglos de Julia.

In [32]:
jarray = nprandom.rand(3,4)
typeof(jarray)

Matrix{Float64} (alias for Array{Float64, 2})

Definamos una función:

In [33]:
objective = x -> cos(x) - x

#15 (generic function with 1 method)

In [34]:
objective(3)

-3.989992496600445

Podemos pasar esta función de Julia a un módulo de Python:

In [35]:
so = pyimport("scipy.optimize")
so.newton(objective, 1)

0.7390851332151607

Adicionalmente, podemos consultar la ayuda de la función `newton()` en Python:

In [36]:
?so.newton


    Find a zero using the Newton-Raphson or secant method.

    Find a zero of the function `func` given a nearby starting point `x0`.
    The Newton-Raphson method is used if the derivative `fprime` of `func`
    is provided, otherwise the secant method is used.  If the second order
    derivative `fprime2` of `func` is provided, then Halley's method is used.

    Parameters
    ----------
    func : function
        The function whose zero is wanted. It must be a function of a
        single variable of the form f(x,a,b,c...), where a,b,c... are extra
        arguments that can be passed in the `args` parameter.
    x0 : float
        An initial estimate of the zero that should be somewhere near the
        actual zero.
    fprime : function, optional
        The derivative of the function when available and convenient. If it
        is None (default), then the secant method is used.
    args : tuple, optional
        Extra arguments to be used in the function call.
    tol : float,

In [37]:
integrate = pyimport("scipy.integrate")

PyObject <module 'scipy.integrate' from '/home/oscar/anaconda3/lib/python3.6/site-packages/scipy/integrate/__init__.py'>

In [38]:
f1(x,t) = -x

f1 (generic function with 1 method)

In [39]:
t = 0:0.1:10

0.0:0.1:10.0

In [40]:
soln = integrate.odeint(f1, 1, t);